In [1]:
import torch
import cv2
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models import efficientnet_b0
from torchvision.transforms import functional as F

# **ResNet-50 Modele backbone**

In [2]:
# Charger le modèle Faster R-CNN avec ResNet-50 comme backbone
model = fasterrcnn_resnet50_fpn(pretrained=True,weights='DEFAULT')
model.eval()  # Mettre le modèle en mode évaluation
print("Modèle avec Eesnet50 comme backbone chargé avec succès.")

Modèle avec Eesnet50 comme backbone chargé avec succès.


In [3]:
def draw_boxes(image, boxes, labels, scores, threshold=0.5):
    for i in range(len(boxes)):
        if scores[i] > threshold:
            box = boxes[i]
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{labels[i]}: {scores[i]:.2f}"
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image

In [4]:
# Charger la vidéo d'entrée
input_video_path = './Video_test/test_04.mp4'
output_video_path = 'output_test_04.mp4'
cap = cv2.VideoCapture(input_video_path)
# Obtenir les propriétés de la vidéo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
# Configurer la vidéo de sortie
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
# Boucle pour lire chaque frame de la vidéo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Convertir l'image en tensor pour le modèle
    image_tensor = F.to_tensor(frame).unsqueeze(0)
    # Faire des prédictions
    with torch.no_grad():
        predictions = model(image_tensor)
    # Extraire les boîtes, les scores et les labels
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    # Dessiner les boîtes englobantes sur la frame
    frame = draw_boxes(frame, boxes, labels, scores, threshold=0.5)
    # Afficher ou sauvegarder la frame avec les détections
    out.write(frame)
    cv2.imshow('Video Detection', frame)
    # Quitter la boucle si 'q' est pressé
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

# **VGG19 Modele Backbone**

In [5]:
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN
from torchvision.models import vgg19

In [6]:
# Charger le modèle VGG19 pré-entraîné
vgg19_model = vgg19(pretrained=True)
vgg19_features = vgg19_model.features
# Ajouter une couche d'adaptation pour correspondre à Faster R-CNN
vgg19_features.out_channels = 512  # Définir le nombre de canaux de sortie

# Définir un générateur d’ancres adapté
anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),  # Différentes tailles d'ancres
    aspect_ratios=((0.5, 1.0, 2.0),)  # Ratios largeur/hauteur
)

# Créer le modèle Faster R-CNN avec le backbone modifié
model = FasterRCNN(
    backbone=vgg19_features,
    num_classes=2,  
    rpn_anchor_generator=anchor_generator
)

# Mode évaluation (désactive le dropout et la normalisation batch)
model.eval()
print("Modèle avec VGG19 comme backbone chargé avec succès.")

c:\Users\ousse\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ousse\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Modèle avec VGG19 comme backbone chargé avec succès.


In [7]:
# Charger la vidéo d'entrée
input_video_path = './Video_test/test_04.mp4'
output_video_path = 'output_test_04_vgg19.mp4'
cap = cv2.VideoCapture(input_video_path)
# Obtenir les propriétés de la vidéo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
# Configurer la vidéo de sortie
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
# Boucle pour lire chaque frame de la vidéo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Convertir l'image en tensor pour le modèle
    image_tensor = F.to_tensor(frame).unsqueeze(0)
    # Faire des prédictions
    with torch.no_grad():
        predictions = model(image_tensor)
    # Extraire les boîtes, les scores et les labels
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    # Dessiner les boîtes englobantes sur la frame
    frame = draw_boxes(frame, boxes, labels, scores, threshold=0.4)
    # Afficher ou sauvegarder la frame avec les détections
    out.write(frame)
    cv2.imshow('Video Detection', frame)
    # Quitter la boucle si 'q' est pressé
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

# **EfficientNet Modele Backbone**

In [8]:
# Charger EfficientNet-B0 avec des poids pré-entraînés
efficientnet = efficientnet_b0(weights='DEFAULT')
backbone = efficientnet.features
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),  
    aspect_ratios=((0.5, 1.0, 2.0),)  
)

# Construire le modèle Faster R-CNN
model_EfficientNet = FasterRCNN(
    backbone=backbone,
    num_classes=2,  # Changez pour votre nombre réel de classes (+1 pour le fond)
    rpn_anchor_generator=anchor_generator
)
model_EfficientNet.eval()
print("Modèle avec EfficientNet comme backbone chargé avec succès.")

Modèle avec EfficientNet comme backbone chargé avec succès.


In [9]:
# Charger la vidéo d'entrée
input_video_path = './Video_test/test_04.mp4'
output_video_path = 'output_test_04_EfficientNet.mp4'
cap = cv2.VideoCapture(input_video_path)
# Obtenir les propriétés de la vidéo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
# Configurer la vidéo de sortie
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
# Boucle pour lire chaque frame de la vidéo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Convertir l'image en tensor pour le modèle
    image_tensor = F.to_tensor(frame).unsqueeze(0)
    # Faire des prédictions
    with torch.no_grad():
        predictions = model_EfficientNet(image_tensor)
    # Extraire les boîtes, les scores et les labels
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    # Dessiner les boîtes englobantes sur la frame
    frame = draw_boxes(frame, boxes, labels, scores, threshold=0.5)
    # Afficher ou sauvegarder la frame avec les détections
    out.write(frame)
    cv2.imshow('Video Detection', frame)
    # Quitter la boucle si 'q' est pressé
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()